In [ ]:
%load_ext lab_black

In [ ]:
import pandas as pd
from zipfile import ZipFile

In [ ]:
# take a look at the data

file_companies_2005_Q1 = "../data/raw/company_2005_Q1.zip"
file_companies_2011_Q4 = "../data/raw/company_2011_Q4.zip"

myzip = ZipFile(file_companies_2005_Q1)
#     print(myzip.namelist())
filestream_2005 = myzip.open("company.idx")
myzip.close()

myzip = ZipFile(file_companies_2011_Q4)
filestream_2011 = myzip.open("company.idx")
myzip.close()

In [ ]:
# load company list data
df_2005 = pd.read_csv(
    filestream_2005,
    sep=r" [ ]+",
    header=4,
    skiprows=[8],
    error_bad_lines=False,
    warn_bad_lines=False,
)
df_2005.info()

In [ ]:
# load company list data
df_2011 = pd.read_csv(
    filestream_2011,
    sep=r" [ ]+",
    header=4,
    skiprows=[9],
    error_bad_lines=False,
    warn_bad_lines=False,
    encoding_errors="replace",
)
df_2011.info()

In [ ]:
# cleaning

def to_int(x):
    try:
        return int(x)
    except (pd.errors.IntCastingNaNError, ValueError):
        return -1


# 2005
df_2005["CIK"] = df_2005["CIK"].apply(lambda x: to_int(x))
df_2005["Date_filed"] = pd.to_datetime(df_2005["Date Filed"])
df_2005["Form Type"] = df_2005["Form Type"].astype("category")

# 2011
df_2011["Date_filed"] = pd.to_datetime(df_2011["Date Filed"])
df_2011["Form Type"] = df_2011["Form Type"].astype("category")

df_2005.to_csv("../data/processed/companies_2005_Q1.csv")
df_2011.to_csv("../data/processed/companies_2011_Q4.csv")

In [ ]:
# select only 10-Q fillings

df_2005_10Q = df_2005[df_2005["Form Type"] == "10-Q"]
df_2011_10Q = df_2011[df_2011["Form Type"] == "10-Q"]
print(df_2005_10Q.shape, df_2011_10Q.shape)

In [ ]:
counts = df_2005_10Q.value_counts("Company Name")
print(sum(counts > 1) / len(counts))
counts

In [ ]:
counts = df_2011_10Q.value_counts("Company Name")
print(sum(counts > 1) / len(counts))
counts

In [ ]:
# What's happening with this company... ?
df_2011_10Q[df_2011_10Q["Company Name"] == "Grupo International Inc"]

In [ ]:
# Select only companies (a bit rough...)

is_company = ["INC", "Inc", "CORP", "LTD", "Corp", "CO", "inc"]
df_2005_10Q_inc = df_2005_10Q[
    df_2005_10Q["Company Name"].str.contains("|".join(is_company))
]
df_2011_10Q_inc = df_2011_10Q[
    df_2011_10Q["Company Name"].str.contains("|".join(is_company))
]

In [ ]:
# Check the remaining entries

df_2005_10Q_fund = df_2005_10Q[
    ~df_2005_10Q["Company Name"].str.contains("|".join(is_company))
]
df_2011_10Q_fund = df_2011_10Q[
    ~df_2011_10Q["Company Name"].str.contains("|".join(is_company))
]
df_2005_10Q_fund
df_2011_10Q_fund

In [ ]:
df_2005_10Q_inc.drop_duplicates("Company Name", inplace=True)
df_2011_10Q_inc.drop_duplicates("Company Name", inplace=True)
print(df_2005_10Q_inc.shape, df_2011_10Q_inc.shape)

In [ ]:
# merge to see what companies listed in 2005 are still there in 2011

df_merge = pd.merge(
    df_2005_10Q_inc,
    df_2011_10Q_inc,
    on="Company Name",
    how="inner",
    validate="one_to_one",
)
print(df_merge.shape)
print(len(df_2011_10Q_inc) - len(df_merge))

Out of 1397 companies listed in 2005, 418 remain in 2011.<br>
Meanwhile, 6259 (!!) companies went public after Q1 2005.

In [ ]:
# save final files to CSV

df_2005_10Q_inc.to_csv("../data/processed/companies_2005_10Q.csv")
df_2011_10Q_inc.to_csv("../data/processed/companies_2011_10Q.csv")